In [3]:
# Importing necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
# Replace 'titanic.csv' with your dataset file path if using a local file
titanic = pd.read_csv('titanic.csv')

# Display the first few rows of the dataset
titanic.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Drop irrelevant columns
titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Display the dataset info
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [6]:
# Impute missing values
# For numerical columns, we use the mean value
# For categorical columns, we use the most frequent value
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

titanic['Age'] = num_imputer.fit_transform(titanic[['Age']])
titanic['Embarked'] = cat_imputer.fit_transform(titanic[['Embarked']])[:, 0] # Extract the first (and only) column

# Verify missing values are handled
titanic.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [7]:
scaler = StandardScaler()

# Apply scaling to numerical columns
titanic[['Age', 'Fare']] = scaler.fit_transform(titanic[['Age', 'Fare']])

# Display normalized data
titanic.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,-0.592481,1,0,-0.502445,S
1,1,1,female,0.638789,1,0,0.786845,C
2,1,3,female,-0.284663,0,0,-0.488854,S
3,1,1,female,0.407926,1,0,0.420730,S
4,0,3,male,0.407926,0,0,-0.486337,S


In [8]:
# Encode categorical features
encoder = OneHotEncoder(drop='first', sparse=False)

encoded_features = pd.DataFrame(encoder.fit_transform(titanic[['Sex', 'Embarked']]),
                                columns=encoder.get_feature_names_out(['Sex', 'Embarked']))

# Drop original categorical columns and concatenate encoded features
titanic.drop(['Sex', 'Embarked'], axis=1, inplace=True)
titanic = pd.concat([titanic, encoded_features], axis=1)

# Display the updated dataset
titanic.head()


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,0,3,-0.592481,1,0,-0.502445,1.0,0.0,1.0
1,1,1,0.638789,1,0,0.786845,0.0,0.0,0.0
2,1,3,-0.284663,0,0,-0.488854,0.0,0.0,1.0
3,1,1,0.407926,1,0,0.420730,0.0,0.0,1.0
4,0,3,0.407926,0,0,-0.486337,1.0,0.0,1.0


In [9]:
# Create a new feature 'FamilySize' by combining 'SibSp' and 'Parch'
titanic['FamilySize'] = titanic['SibSp'] + titanic['Parch'] + 1

# Drop 'SibSp' and 'Parch' as they are now represented by 'FamilySize'
titanic.drop(['SibSp', 'Parch'], axis=1, inplace=True)

# Display the updated dataset
titanic.head()


,Survived,Pclass,Age,Fare,Sex_male,Embarked_Q,Embarked_S,FamilySize
0,0,3,-0.592481,-0.502445,1.0,0.0,1.0,2
1,1,1,0.638789,0.786845,0.0,0.0,0.0,2
2,1,3,-0.284663,-0.488854,0.0,0.0,1.0,1
3,1,1,0.407926,0.420730,0.0,0.0,1.0,2
4,0,3,0.407926,-0.486337,1.0,0.0,1.0,1
